# Project 3: Web APIs and NLP <br>

#### Brandie Hatch

## Problem



Learn with Chewie presents:
Web API and NLP Services

## Preparation

In [18]:
# Gather Data
import praw

reddit = praw.Reddit(
    client_id = '5aTMVLqpXQvo6vpgNv171w',
    client_secret = 'FAjO19Zyx5W4N9llEeklP8apwz20Zw',
    username='BrandieHatch',
    password='Qwert123',
    user_agent='praw_BH',
)
print(reddit.read_only)

False


In [19]:
subreddit = reddit.subreddit('Dogtraining')

print(subreddit.display_name)
print(subreddit.title)
print(subreddit.description)

Dogtraining
DogTraining: For training and rehabilitating dogs and puppies and all things canine behavior
###Welcome
This is a forum on dog training and behavior that focuses on a [least intrusive, minimally aversive approach.](https://iaabc.org/about/LIMA)    
   
The advice here is not a replacement for professional help. If your training is not fun and effective, or if you need additional help, then please [find a certified trainer](http://www.reddit.com/r/Dogtraining/wiki/findingatrainer) for assistance.  
  
----   
  
###Filters   
[All | Remove Filter](http://reddit.com/r/Dogtraining/)   
[Academic](http://ac.reddit.com/r/dogtraining/search?q=flair%3Aacademic&restrict_sr=on&sort=new&t=all#ac) [Announcement](http://an.reddit.com/r/Dogtraining/search?q=flair%3Aannouncing&restrict_sr=on&sort=new&t=all#an)  
[Brags](http://br.reddit.com/r/Dogtraining/search?q=flair%3Abrags&restrict_sr=on&sort=new&t=all#br) [Community](http://cm.reddit.com/r/Dogtraining/search?q=flair%3Acommunity&rest

In [20]:
hot_dogtraining = subreddit.hot(limit=1)

for submission in hot_dogtraining:
    print(dir(submission))   # can see what all is connected

['STR_FIELD', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_chunk', '_comments_by_id', '_fetch', '_fetch_data', '_fetch_info', '_fetched', '_kind', '_reddit', '_reset_attributes', '_safely_add_arguments', '_url_parts', '_vote', 'all_awardings', 'allow_live_comments', 'approved_at_utc', 'approved_by', 'archived', 'author', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_template_id', 'author_flair_text', 'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_is_blocked', 'author_patreon_flair', 'author_premium', 'award', 'awarders', 'banned_at_utc', 'banned_by', 'can_gild', 'can_mod_post', 'category', 'clear_

In [21]:
hot_dogtraining = subreddit.hot(limit=5)

for submission in hot_dogtraining:
    print(submission.ups)   
    print(submission.downs)
    print(submission.title)

21
0
Save the Date! - Upcoming major dog training event list for 2022 Apr - 2022 Sep
13
0
Trick of the Month - April 2022 - Balance on an Object
36
0
My overseas rescue dog is agoraphobic and doesn't know how to walk on a leash - advice, please?
1521
0
3 month old lab is my 4 year olds first puppy.
41
0
Dog won’t stop ripping up furniture in the middle of the night


In [22]:
hot_dogtraining = subreddit.hot(limit=5)

for submission in hot_dogtraining:
    if not submission.stickied:
        print('Title: {}, ups: {}, downs: {}'.format(submission.title, submission.ups, submission.downs))

Title: My overseas rescue dog is agoraphobic and doesn't know how to walk on a leash - advice, please?, ups: 41, downs: 0
Title: 3 month old lab is my 4 year olds first puppy., ups: 1525, downs: 0
Title: Dog won’t stop ripping up furniture in the middle of the night, ups: 39, downs: 0


## Explore Data

In [2]:
# Imports



## Model

## Model Evaluation

## Answer Problem, Conclusions, and Recommendations

## Sources